In [ ]:
import pandas as pd
import numpy as np


#data from : https://www.kaggle.com/columbia2131/speed-up-reading-csv-to-pickle/notebook
train = pd.read_pickle('../input/ump-train-picklefile/train.pkl')

In [ ]:
features = ['investment_id']
features+=(f'f_{i}' for i in range(300))
target=train['target']
all_data=train[features]

In [ ]:
def counts(all_data) :
    
    from tqdm import tqdm

    num_cols = [c for c in all_data.columns]

    for f in tqdm(num_cols):
        all_data[f+'_counts'] = all_data[f].value_counts()
        all_data[f+'_counts'] = all_data[f+'_counts'].fillna(1)
    return all_data

In [ ]:
all_data=counts(all_data)

In [ ]:
y_val=target[:4000]
dvalid=all_data[:4000]
all_data=all_data[4000:]
target=target[4000:]


In [ ]:
from catboost import CatBoostRegressor
model = CatBoostRegressor()
model.fit(all_data, target,
             use_best_model=True,
             verbose=50)
val_preds = model.predict(dvalid)
from sklearn.metrics import mean_absolute_error 
MAEValue = mean_absolute_error(y_val, val_preds, multioutput='uniform_average') 
print(f"mean_absolute_error : {MAEValue}")

In [ ]:

import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

In [ ]:

for (test_df, sample_prediction_df) in iter_test:
    test_df=test_df[features]
    test_df=counts(test_df)

    preds = model.predict(test_df.values)
    
    sample_prediction_df['target'] = preds  # make your predictions here
    env.predict(sample_prediction_df)  